## What is Python Data Model?
*We can think of data model as a description of Python as a framework.It formalizes the interfaces of the building blocks of the language itself.We can see Python’s data model as a powerful API you can interface with by implementing one or more dunder methods.  --  Luciano Ramalho, Fluent Python*

If we want to write more Pythonic code, knowing how and when to use special methods is an important step.

Special(Magic/dunder) methods are part of Python Data Model, the special methods are a set of predefined methods you can use to enrich your classes. They are easy to recognize because they start and end with double underscores, for example __init__ or __str_. Special methods allow us to make any customed class more Pythonic!

Let's demonstrate the power of special methods by implementing a simple Card Deck Class.

In [ ]:
import collections
from random import choice, shuffle


Card = collections.namedtuple('Card','rank suit')

class FrenchDeck:
    
    ranks = [str(n) for n in range(2,11)]+list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank,suit)for suit in self.suits for rank in self.ranks]  

deck = FrenchDeck()

The FrenchDeck class has two class properties ranks: 2, 3,4 till JQKA and suits: spades, diamonds, clubs, hearts. We use namedtuple to construct a simple class to represent invidual cards as following:

In [ ]:
Card('7','diamonds')

The deck is an instance of our customed class

In [ ]:
type(deck)

The content of the deck object is a list of tuples

In [ ]:
deck._cards

The point of this example is the FrenchDeck class, the deck object contains a list tuples, can we perform common list operations on our created object like a regular python list?

Common operations on a Python list:
- print
- len
- selection
- slicing
- sorting
- random selection
- random shuffle

In [ ]:
print(deck)

In [ ]:
len(deck)

In [ ]:
deck[0]

In [ ]:
sorted(deck)

In [ ]:
for d in deck:
    print(d)

In [ ]:
shuffle(deck)

In contrast, let's craete a object of standard python list type

In [ ]:
demo_list = [tuple([i,j]) for i,j in zip(range(10),'abcdefghijk')]

common_operations_check(demo_list,(0, 'a'))

In [ ]:
demo_list

We see that we are able to do print, selection, sorting, shuffle fine in this demo_list.

In [ ]:
print (demo_list)

In [ ]:
len(demo_list)

In [ ]:
demo_list[:2]

In [ ]:
shuffle(demo_list)

The goal is to enable our FrenchDeck class to be able to emulates the above standard python operations.

We write the following functions to check if a python collection object like deck emulates the behavior of built-in types.

In [ ]:
import inspect,re
from IPython.display import Markdown

def red_print(message):
    display (Markdown('<span style="color: #ff0000">'+message+'</span>'))


def safe_run(func):
    def func_wrapper(*args, **kwargs):
        func_name = str(inspect.stack()[1].code_context[0]).replace('\n','')
        try:
            print (func_name)
            return func(*args, **kwargs)
        except Exception as e:
            fail_message = 'failed'
            red_print(fail_message+str(e))
            return None
    return func_wrapper


@safe_run
def len_check(deck):
    print(len(deck))

@safe_run
def selection_check(deck):
    print(deck[0])
    
@safe_run
def slicing_check(deck):
    print(deck[::2])
    
@safe_run
def isin_check(deck,instance):
    print(instance in deck)
    
@safe_run
def sort_check(deck):
    print(sorted(deck))
    
@safe_run 
def forloop_check(deck):
     for i in deck:
        print (i)
        
@safe_run    
def choice_check(deck):
    print (choice(deck))
    
@safe_run    
def shuffle_check(deck):
    shuffle(deck)
    print (deck)
        
def common_operations_check(deck,instance):
    
    print(deck,'\n')
    len_check(deck) 
    selection_check(deck)
    isin_check(deck,instance)
    sort_check(deck)
    forloop_check(deck)
    choice_check(deck)
    shuffle_check(deck)


Let's try to apply common operations on a built-in list.

In [ ]:
common_operations_check(demo_list,(4, 'e'))

Next let's try to perform above operations on the deck object

In [ ]:
common_operations_check(deck,Card('Q','hearts'))

### Lots of failures!! 
Before using special methods to resolve the failures, let's look at an Bad example of implementing common methods in customed class

In [ ]:
class BadFrenchDeck:
    
    ranks = [str(n) for n in range(2,11)]+list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank,suit)for suit in self.suits for rank in self.ranks]  
    
    def show(self):
        return 'FrenchDeck(%r)'%(self._cards)
    
    def length(self):
        return len(self._cards)
    
    def selection(self,index):
        return (self._cards[index])
    
    def slicing(self,indices):
        return (self._cards[indices])
    
    def deck_sort(self):
        return sorted(self._cards)
    
    def is_in_deck(self, instance):
        return instance in self._cards
        
    
    ### shuffle, random choice ? 
    
baddeck = BadFrenchDeck()
print (baddeck.show())
print (baddeck.length())
print (baddeck.selection(1))
baddeck.deck_sort()
print (baddeck.is_in_deck(Card(rank='2', suit='spades')))

## What are the problems of the above approaches??

- the users of your class have to memorize arbitary method names for standard operations("how to get the numbers of items?) is it .size(), .length() or what? 
- recreating wheels: lots of work and hacks

## Python Data Model Approach
While coding with any framework, you spend lots of time implementing methods that are called by the framework. The same happens when we leverage the python data model, the python interpreter invokes special methods to perform basic object operations, often triggered by special syntax.

The special method names are always written with leading and trailing double underscores(i.e.,__getitem__). the syntax object[key] is supported by the __getitem__ special method, in order to evaluate my_collection[key], the interpreter calls my_collection.__getitem__(key).
### special method1: object.__ repr __ 

In [ ]:
Card = collections.namedtuple('Card','rank suit')

class FrenchDeck:
    
    ranks = [str(n) for n in range(2,11)]+list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank,suit)for suit in self.suits for rank in self.ranks]  
    
    def __repr__(self):
        return 'FrenchDeck(%r)'%(self._cards)
    

    
deck = FrenchDeck()
common_operations_check(deck,Card('Q','hearts'))

Called by the repr() built-in function to compute official string representation of an object. If at all possible, this should look like a valid Python expression that could be used to recreate an object with the same value. Another similar special method is object.__str__. The default implementation defined by the built-in type object calls object.__repr__().
tr() is used for creating output for end user while repr() is mainly used for debugging and development. repr’s goal is to be unambiguous and str’s is to be readable. For example, if we suspect a float has a small rounding error, repr will show us while str may not.


### special method2: object.__ len __ 

In [ ]:
Card = collections.namedtuple('Card','rank suit')

class FrenchDeck:
    
    ranks = [str(n) for n in range(2,11)]+list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank,suit)for suit in self.suits for rank in self.ranks]  
    
    def __repr__(self):
        return 'FrenchDeck(%r)'%(self._cards)
    
    def __len__(self):
        return len(self._cards)
    
deck = FrenchDeck()
common_operations_check(deck,Card('Q','hearts'))
    

### Why len is not a mehod?
IF you learnt another OO language before Python, you may have found it strange to use len(collection) instead of collection.len().

Let's look at Cpython source code

https://github.com/python/cpython/blob/92c7e30adf5c81a54d6e5e555a6bdfaa60157a0d/Python/bltinmodule.c#L1536-L1556

and definition of PyObject in c-api documention:
https://docs.python.org/3/c-api/structures.html#c.PyObject

No method is called for the built-in objects of CPython: the length is simply read from a field in C struct. len() is not called as a special method because it gets special treatment as part of Python Data Model, just like abs. Thanks for the special method __ len __,we can make len work with our own custom objects.

We have just seen two advantages of using special methods to leverage the Python data model: 1. the users of your class don't have to memorize arbitary method names for standard operations("how to get the numbers of items?) is it .size(), .length() or what? 2. it is easier to benefit from the rich python standard library and avoid reinventing the wheel, like th random.choice function. It gets better, since our __getitem__ delegates to the [] operator of self._cards, our deck automatically support slicing.

### special method3: object.__ getitem __ 

In [ ]:
Card = collections.namedtuple('Card','rank suit')

class FrenchDeck:
    
    ranks = [str(n) for n in range(2,11)]+list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank,suit)for suit in self.suits for rank in self.ranks]  
    
    def __repr__(self):
        return 'FrenchDeck(%r)'%(self._cards)
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self,position):
        return self._cards[position]
    
    
deck = FrenchDeck()
common_operations_check(deck,Card('Q','hearts'))

In [ ]:
deck[::2]

In [ ]:
for d in deck:
    print (d)

object.__getitem__(self, key):
Called to implement evaluation of self[key]. For sequence types, the accepted keys should be integers and slice objects. Note that the special interpretation of negative indexes (if the class wishes to emulate a sequence type) is up to the __getitem__() method. 

iter():
Return an iterator object. The first argument is interpreted very differently depending on the presence of the second argument. Without a second argument, object must be a collection object which supports the iteration protocol (the __iter__() method), or it must support the sequence protocol (the __getitem__() method with integer arguments starting at 0). If it does not support either of those protocols, TypeError is raised.

### special method4: object.__ setitem __ 

In [ ]:
Card = collections.namedtuple('Card','rank suit')

class FrenchDeck:
    
    ranks = [str(n) for n in range(2,11)]+list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank,suit)for suit in self.suits for rank in self.ranks]  
    
    def __repr__(self):
        return 'FrenchDeck(%r)'%(self._cards)
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self,position):
        return self._cards[position]
    
    def __setitem__(self, key, value):
        self._cards[key] = value
        
deck = FrenchDeck()
common_operations_check(deck,Card('Q','hearts'))

In [ ]:
deck[:2]

In [ ]:
sorted(deck)

object.__setitem__(self, key, value)¶
Called to implement assignment to self[key]. Same note as for __getitem__(). This should only be implemented for mappings if the objects support changes to the values for keys, or if new keys can be added, or for sequences if elements can be replaced. The same exceptions should be raised for improper key values as for the __getitem__() method.

reversed(seq)
Return a reverse iterator. seq must be an object which has a __reversed__() method or supports the sequence protocol (the __len__() method and the __getitem__() method with integer arguments starting at 0).

In [ ]:
shuffle(deck)

In [ ]:
deck

In [ ]:
for card in reversed(deck[:5]):
    print (card)

Python Data Model Lego Block Illustration

Without Special Methods
<img src="no_fit.png">

With special methods
<img src="data_model.png">

## Summary
By implmenting special methods len and getitem, our frenchdeck behaves like a standard python sequence, allowing it to benefit from core language features e.g. iteration and slicing from standard library: random.choice, reversed, sorted.

### 1. Implementing special/dunder methods to make objects behave like the built-in types, rather than reinvieting wheels

### 2. Special methods allow objects to benefit from the rich Python standard library

### 3. Special Methods are not meant to be called directly, use built-in functions instead
- Special methods are used is that they are meant to be called by the python interpreter, not by us, unless we are doing metaprogramming, the only special method that is frequently called by user code directly is __init__, to invoke the superclass in your own __init__ implementation. We don't write my_object.__len__(). we write len(my_object), and if my_object is an instance of a user-defined class, then Python calls the __len__ instance method we implemented.
- These built-ins call the corresponding special method, and interpreter takes a shortcut: the cpython implementation of len() actually returns the value of the ob_size field in PyVarObject C struct that represents any variable-sized built-in object in memory, this is much faster than calling a method.
- The special method call is implicit, for example, the statement for i in x: actually causes the invocations of iter(x), which in turn may call x.__iter__() if that is available. If we need to invoke a special method, it is usually better to call the related built-in function(e.g,; len, iter, str,etc).
   

built-in function invokes  special method
    repr()         -->     __repr__ ;
    str()          -->     __str__ ;
    len()          -->     __len__ ;
    iter()         -->     __getitem__ or __iter__ ;
    reversed()     -->     __reversed__ or (__len__ and __getitem__) ;
    self[key]      -->     __getitem__ (evaluation) ;
    self[key]      -->     __setitem__ (assignment) ;

Further reading
The data model of Python Language Refendce is the canonical source:
https://docs.python.org/3/reference/datamodel.html
